Event,Lock,Condition,Barrier,Semaphore 它们各有各的特点和适用场景！但是它们共同的东西就是说它们都是用于线程同步的，处理线程间的相互关系

In [1]:
import time 
import logging
import threading
import importlib

importlib.reload(logging)
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s  [%(threadName)s] %(message)s')

In [2]:
logging.info('xixi')

2017-05-03 20:07:57,802 INFO  [MainThread] xixi


## 线程之间的通讯

In [7]:
import queue     ## 引入一个标准库 queue(队列)     这是更能接受的一种模式
import random

In [ ]:
## 再重新改写生产者、消费者模型

In [6]:
def producer(queue: queue.Queue, event: threading.Event):
    while not event.wait(3):    ## 每3秒产生一个数据
        data = random.randint(0,100)
        logging.info(data)
        queue.put(data)        ## put 进来

In [11]:
def consumer(queue: queue.Queue, event: threading.Event):
    while not event.is_set():
        logging.info(queue.get())

In [12]:
q = queue.Queue()      ## 初始化一个q 

In [13]:
e = threading.Event()  ## 初始化一个event

In [14]:
threading.Thread(target=consumer, name='consumer',args=(q,e)).start()       ## 生产者、消费者通常都是先启动消费者

In [ ]:
## 执行上面后，发现并没有输出任何东西

In [15]:
threading.Thread(target=producer, name='producer', args=(q,e)).start()  ## 生产一条数据，就消费一条数据 

2017-05-03 20:20:06,712 INFO  [producer] 84
2017-05-03 20:20:06,741 INFO  [consumer] 84
2017-05-03 20:20:09,742 INFO  [producer] 69
2017-05-03 20:20:09,770 INFO  [consumer] 69
2017-05-03 20:20:12,771 INFO  [producer] 91
2017-05-03 20:20:12,772 INFO  [consumer] 91
2017-05-03 20:20:15,773 INFO  [producer] 30
2017-05-03 20:20:15,775 INFO  [consumer] 30
2017-05-03 20:20:18,775 INFO  [producer] 41
2017-05-03 20:20:18,778 INFO  [consumer] 41
2017-05-03 20:20:21,778 INFO  [producer] 3
2017-05-03 20:20:21,800 INFO  [consumer] 3


In [16]:
e.set()  ## 需要停止的时候，e.set()就好了

队列是先进先出的容器

它做为一个容器，当生产者、消费者速率不一致的时候，它是可以暂存的，这是它的一个优势。

如果用 Condition 实现的话，两边必须是同步的！生产一条必须消费一条。

它的劣势就是无法广播的！

所以，通常情况下，queue和Condition会一起使用；queue可以实现暂存，Condition 可以实现广播。
所以，有暂存和广播需求的时候，可以同时配合使用。

In [ ]:
queue.Queue   ##  这个常用的类就实现了先进先出

In [ ]:
queue.PriorityQueue   ## 优先队列

In [39]:
help(queue.Queue)

Help on class Queue in module queue:

class Queue(builtins.object)
 |  Create a queue object with a given maximum size.
 |  
 |  If maxsize is <= 0, the queue size is infinite.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, maxsize=0)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  empty(self)
 |      Return True if the queue is empty, False otherwise (not reliable!).
 |      
 |      This method is likely to be removed at some point.  Use qsize() == 0
 |      as a direct substitute, but be aware that either approach risks a race
 |      condition where a queue can grow before the result of empty() or
 |      qsize() can be used.
 |      
 |      To create code that needs to wait for all queued tasks to be
 |      completed, the preferred technique is to use the join() method.
 |  
 |  full(self)
 |      Return True if the queue is full, False otherwise (not reliable!).
 |      
 |      This method is likely to be removed at some point.  Use qsiz

In [17]:
q = queue.Queue(maxsize=5)   ## 可选的参数maxsize ，队列最长能到多长

In [18]:
## 两个方法：  put  和 get 

In [ ]:
q.put 和 q.put_nowait     ## 后者是前者的一个特例

In [19]:
help(q.put)  ## 接收3个参数  item就是put到队列里的元素 

Help on method put in module queue:

put(item, block=True, timeout=None) method of queue.Queue instance
    Put an item into the queue.
    
    If optional args 'block' is true and 'timeout' is None (the default),
    block if necessary until a free slot is available. If 'timeout' is
    a non-negative number, it blocks at most 'timeout' seconds and raises
    the Full exception if no free slot was available within that time.
    Otherwise ('block' is false), put an item on the queue if a free slot
    is immediately available, else raise the Full exception ('timeout'
    is ignored in that case).



In [20]:
## 之前配置的q = queue.Queue(maxsize=5)   所以我们put 5 次

for x in range(5):
    q.put(x)

In [21]:
q.qsize()   ## qsize方法返回当前queue的尺寸

5

In [22]:
q.put(6)   ## block为True ，timout为None 的时候 它会阻塞   手动中断

KeyboardInterrupt: 

In [23]:
q.put(6,False)  ## block设置为False ，它不阻塞，会抛出Full异常

Full: 

In [24]:
q.put(6, timeout=3)  ## 阻塞直到timeout到期，并抛出 queue.Full 异常

Full: 

注：如果队列不满的时候，它没有任何问题！

In [25]:
help(q.put_nowait)   ## 就一个item参数

Help on method put_nowait in module queue:

put_nowait(item) method of queue.Queue instance
    Put an item into the queue without blocking.
    
    Only enqueue the item if a free slot is immediately available.
    Otherwise raise the Full exception.



In [26]:
q.put_nowait(6)   ## 等价于 q.put(6, False)  block为False

Full: 

### put是往队列里加元素，而get是从队列里取元素。

In [27]:
q.get()

0

In [28]:
q.get()

1

In [29]:
q.get()    ## 它是一个元素一个元素的取出来

2

In [30]:
q.get()

3

In [31]:
q.get()

4

In [32]:
q.qsize()    ## 5 个元素都取出来了，里面没有元素了

0

In [ ]:
## 当没有元素的时候

In [33]:
q.get()    ## 阻塞住了，直到里面有元素为止  get的行为跟put是对应起来的 

KeyboardInterrupt: 

In [34]:
q.get(False)  ## 直接抛出 Empty 异常    raise queue.Empty

Empty: 

In [35]:
help(q.get)

Help on method get in module queue:

get(block=True, timeout=None) method of queue.Queue instance
    Remove and return an item from the queue.
    
    If optional args 'block' is true and 'timeout' is None (the default),
    block if necessary until an item is available. If 'timeout' is
    a non-negative number, it blocks at most 'timeout' seconds and raises
    the Empty exception if no item was available within that time.
    Otherwise ('block' is false), return an item if one is immediately
    available, else raise the Empty exception ('timeout' is ignored
    in that case).



In [36]:
q.get(timeout=3)   ## 阻塞，直到超时，抛出 queue.Empty 异常

Empty: 

In [37]:
q.get_nowait()   ## 等价于  q.get(False)

Empty: 

queue通常就像以上这些用法

但还有一些其它的方法

In [ ]:
## 比如：

q.maxsize  等等

重点说下： queue.Queue是线程安全的！所以能正确处理并发。

所谓的线程安全就是它做为一个临界资源，我们不需要加锁保护，也能正确的处理线程并发。

Python有一个 全局解释器锁 GIL  的存在

所以，内置容器(list、字典等)都是线程安全的！

当然了，这里指的线程安全是指操作内置容器里的元素是线程安全的，但是操作它整个变量的时候并不是线程安全的。

所谓的全局解释器锁，也就是说在解释器级别加了一把大锁，在操作内置容器的时候加了一把大锁。所以它就线程安全了。

Python 争议比较大的就是全局解释器锁了。

为什么呢？因为 GIL 造成了并发效率低。所以开发时候使用非常舒服，但是效率不高。

In [38]:
import collections  ## 这个库也是线程安全的

### 线程安全的，可以在多线程中放心使用。

其实用的最多的就是queue了，先进先出。

先进先出更多的用在生产者消费者模型，而这是使用非常非常广泛的模型，基本上所有的数据流处理都可以(套上)生产者消费者模型。

queue是无法持久化的，这也是进程内的queue，并不是进程间的。